Jupyter Notebook For ReadMyMind, A CS 125 @ Illinois MP7 Project by Isaac Park and Mihir Pandya.

In [54]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import pprint
import numpy as np



username = 'realDonaldTrump' # working example: Donald Trump

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)
    
tweets_df = pd.DataFrame({
    'Timestamp': (),
    'Likes': (),
    'Retweets': (),
    'Text': (),
    'Sentences': (),
    'Sentiment_Total': (),
    'Keywords': ()
})

tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]
    
recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
for status in recent_tweets:
    test = status.full_text
    if test[:2] != 'RT': # removing retweets made by the user
        status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                index=['Timestamp', 'Likes', 'Retweets', 'Text'])
    tweets_df = tweets_df.append(status_data, ignore_index = True)
    
tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
tweets_df = tweets_df.astype('object')

keywords_dict = {}

for i in range(len(tweets_df)):
    content = tweets_df.iloc[i]['Text']
    if 'http' in content:
        j = content.index('http')
        content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
    content = content.replace('\n', '')
    tweets_df.iloc[i]['Text'] = content
    
    blob = TextBlob(content)
    tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
    sentiments = {}
    
    for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
        sentiments[str(sent)] = sent.sentiment.subjectivity
        
    tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe
    
    tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime
    
    # Keyword extraction goes here
    filtered_words = blob.noun_phrases
    temp = []
    
    for element in filtered_words:
        for x in range(len(filtered_words)):
            if element != filtered_words[x] and element in filtered_words[x]:
                temp.append(element)
    parts_of_speech = blob.tags
    for element in temp:
        filtered_words = [x for x in filtered_words if x != element]
    
    for x in range(len(parts_of_speech)):
        if (parts_of_speech[x])[1] == 'NN':
            enter = True
            for element in filtered_words:
                if (parts_of_speech[x])[0] in element:
                    enter = False
            if enter:
                if x > 0 and (parts_of_speech[x - 1])[1] == 'PRP$':
                    filtered_words.append((parts_of_speech[x])[0])
    parenthesis = []
    paren_init = 0
    loc_begin = blob.find("(", paren_init)
    loc_end = blob.find(")", paren_init)
    
    while loc_end >= 0:
        parenthesis.append(blob[loc_begin:loc_end])
        paren_init = loc_end + 1
        loc_begin = blob.find("(", paren_init)
        loc_end = blob.find(")", paren_init)
    
    for element in filtered_words:
        for pelement in parenthesis:
            if element in pelement.lower():
                filtered_words = [x for x in filtered_words if x != element]        
    
    for word in filtered_words:
        separated = TextBlob(word).words
        for j in separated:
            j = word.strip()
            tb = ((TextBlob(j).tags)[0])[1]
            if j.isalpha() and len(j) > 2 and (tb == 'NN' or tb == 'NNS' or tb == 'VBP'):
                if j in keywords_dict:
                    keywords_dict[j][0] += 1
                    keywords_dict[j][1] += tweets_df.iloc[i]['Likes']
                    keywords_dict[j][2] += tweets_df.iloc[i]['Retweets']
                    keywords_dict[j][3] += tweets_df.iloc[i]['Sentiment_Total']
                else:
                    keywords_dict[j] = [1, tweets_df.iloc[i]['Likes'], tweets_df.iloc[i]['Retweets'], tweets_df.iloc[i]['Sentiment_Total']]


for key in keywords_dict:
    keywords_dict[key][1] = int(keywords_dict[key][1] / keywords_dict[key][0])
    keywords_dict[key][2] = int(keywords_dict[key][2] / keywords_dict[key][0])
    keywords_dict[key][3] = keywords_dict[key][3] / keywords_dict[key][0]

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.columns = ['Frequency', 'Avg. Likes', 'Avg. Retweets', 'Avg. Sentiment']
keywords_df.index.name = 'Keywords'
keywords_df.reset_index(inplace = True)
keywords_df = keywords_df.sort_values(['Frequency'], ascending = [False], na_position = 'last')
keywords_df = keywords_df[:30]

keywords_df



,Keywords,Frequency,Avg. Likes,Avg. Retweets,Avg. Sentiment
2,america,10,77871,19453,0.642516
0,fake,9,95661,24409,0.601653
11,pensacola,8,60784,13173,0.348522
26,cnn,7,113715,29318,0.537169
36,florida,6,68073,15662,0.390126
3,market,6,98315,21845,0.527245
5,economy,4,62120,14806,0.294523
56,whitehouse,4,48803,11077,0.440833
32,alabama,4,86608,22229,0.615575
79,cuts,4,94950,21501,0.616493


Proposed method for keyword extraction:

1. Tokenize each word with part of speech tag. keep only proper nouns, nouns, adjectives, and verbs.
2. Score the nouns and proper nouns based on amount of surrounding adjectives and verbs (using more description tends to indicate importance).
3. Record frequency of each word; only keep words that occur above a certain number of times (frequency threshold). These will be our "keywords".
4. Put the list of keywords for each tweet into the 'Keywords' column of the dataframe.

Ideas for graphing the keywords/frequency/likes/retweets relationships:

1. y-axis: frequency, x-axis: keyword; simple bar graph of the top keywords

2. y-axis: likes/retweet count, x-axis: frequencies of keywords; scatter plot with each dot representing a keyword.

3. Simple pie chart to analyze the main content areas that said Twitter account comments on.




In [53]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)



labels = keywords_df['Keywords'][0:10]
values = keywords_df['Frequency'][0:10]
traces = []

trace = go.Pie(labels = labels, values = values, hoverinfo = 'label+percent+name')
traces.append(trace)

layout = go.Layout(height = 600,
                   width = 600,
                   autosize = False,
                   title = "User's Most Covered Topics")
fig = go.Figure(data = traces, layout = layout)
py.iplot(fig, show_link = False)

In [52]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('1mihirpandya', 'CN0KZbwKVr0a5wKrTebB')
data = [go.Bar(
            x=keywords_df['Keywords'],
            y=keywords_df['Frequency']
    )]

py.iplot(data, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~1mihirpandya/0 or inside your plot.ly account where it is named 'basic-bar'


In [63]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd


# Create a trace
trace = go.Scatter(
    name = keywords_df['Keywords'],
    x = keywords_df['Frequency'],
    y = keywords_df['Avg. Retweets'],
    mode = 'markers',
    #hoverinfo = 'name'
)

data = [trace]

py.iplot(data, filename='basic-scatter')